In [21]:
import flwr as fl
import tensorflow as tf
from tensorflow import keras
from keras import layers
import utils as ut
import numpy as np
from typing import Dict, Optional, Tuple

In [22]:
def get_model():
    model = tf.keras.models.Sequential(
        [
            layers.Input(shape=(8,)),  
            layers.Dense(64, activation='relu'),
            layers.Dense(64, activation='relu'),  
            layers.Dense(1)  
        ]
    )
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])
    return model

In [23]:
class CifarClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_test, y_test):
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test

    def fit(self, parameters, config):
        """Train parameters on the locally held training set."""

        # Update local model parameters
        self.model.set_weights(parameters)

        # Get hyperparameters for this round
        batch_size: int = config["batch_size"]
        epochs: int = config["local_epochs"]

        # Train the model using hyperparameters from config
        history = self.model.fit(
            self.x_train,
            self.y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_split=0.2,
        )

        # Return updated model parameters, number of examples trained, and results
        parameters_prime = self.model.get_weights()
        num_examples_train = len(self.x_train)
        results = {
            "mse": history.history["mse"][-1],  # Use the last epoch's MSE
            "mae": history.history["mae"][-1],  # Use the last epoch's MAE
            "val_mse": history.history["val_mse"][-1],  # Use the last epoch's validation MSE
            "val_mae": history.history["val_mae"][-1],  # Use the last epoch's validation MAE
        }

        return parameters_prime, num_examples_train, results

    def get_parameters(self):
        """Get the current parameters of the local model."""
        return self.model.get_weights()

    def evaluate(self, parameters, config):
        """Evaluate parameters on the locally held test set."""
        self.model.set_weights(parameters)
        loss = tf.keras.losses.mean_squared_error(self.y_test, self.model.predict(self.x_test)).numpy().mean().item()
        mae = tf.keras.metrics.mean_absolute_error(self.y_test, self.model.predict(self.x_test)).numpy().mean().item()
        print("*************LOSS******************",loss)
        print("************MAE********************",mae)
        return loss, len(self.x_train), {"mae": mae}

    def get_weights(self):
        """Get the current weights of the local model."""
        return self.model.get_weights()

    def set_weights(self, weights):
        """Set the weights of the local model."""
        self.model.set_weights(weights)

In [24]:
(x_train, y_train), (x_test, y_test) = ut.partition_dataset(2,4,3)

client = CifarClient(get_model(), x_train, y_train, x_test, y_test)

history = fl.client.start_numpy_client(
server_address="127.0.0.1:8080",
client=client,
)

INFO flwr 2023-11-23 01:51:37,689 | grpc.py:49 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-11-23 01:51:37,753 | connection.py:42 | ChannelConnectivity.IDLE
DEBUG flwr 2023-11-23 01:51:37,756 | connection.py:42 | ChannelConnectivity.READY


129/129 [==============================] - 0s 1ms/step
*************LOSS****************** 2.1078832149505615
************MAE******************** 1.1092127561569214
Epoch 1/100
26/26 [==============================] - 1s 13ms/step - loss: 0.5119 - mae: 0.5156 - mse: 0.5119 - val_loss: 0.5861 - val_mae: 0.5145 - val_mse: 0.5861
Epoch 2/100
26/26 [==============================] - 0s 3ms/step - loss: 0.4972 - mae: 0.5025 - mse: 0.4972 - val_loss: 0.5456 - val_mae: 0.5295 - val_mse: 0.5456
Epoch 3/100
26/26 [==============================] - 0s 3ms/step - loss: 0.4874 - mae: 0.4965 - mse: 0.4874 - val_loss: 0.5437 - val_mae: 0.5199 - val_mse: 0.5437
Epoch 4/100
26/26 [==============================] - 0s 3ms/step - loss: 0.4836 - mae: 0.4939 - mse: 0.4836 - val_loss: 0.5578 - val_mae: 0.5087 - val_mse: 0.5578
Epoch 5/100
26/26 [==============================] - 0s 3ms/step - loss: 0.4888 - mae: 0.5015 - mse: 0.4888 - val_loss: 0.5435 - val_mae: 0.5141 - val_mse: 0.5435
Epoch 6/100
26/26 [

DEBUG flwr 2023-11-23 01:53:46,386 | connection.py:139 | gRPC channel closed
INFO flwr 2023-11-23 01:53:46,388 | app.py:215 | Disconnect and shut down


In [25]:
(x_train, y_train), (x_test, y_test) = ut.partition_dataset(0,4,3)

client = CifarClient(get_model(), x_train, y_train, x_test, y_test)

history = fl.client.start_numpy_client(
server_address="127.0.0.1:8081",
client=client,
)

INFO flwr 2023-11-23 01:55:11,181 | grpc.py:49 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-11-23 01:55:11,200 | connection.py:42 | ChannelConnectivity.IDLE
DEBUG flwr 2023-11-23 01:55:11,202 | connection.py:42 | ChannelConnectivity.READY


Epoch 1/50
26/26 [==============================] - 1s 12ms/step - loss: 0.4473 - mae: 0.4716 - mse: 0.4473 - val_loss: 0.3808 - val_mae: 0.4259 - val_mse: 0.3808
Epoch 2/50
26/26 [==============================] - 0s 3ms/step - loss: 0.3332 - mae: 0.4038 - mse: 0.3332 - val_loss: 0.3563 - val_mae: 0.4127 - val_mse: 0.3563
Epoch 3/50
26/26 [==============================] - 0s 3ms/step - loss: 0.3181 - mae: 0.3898 - mse: 0.3181 - val_loss: 0.3508 - val_mae: 0.4046 - val_mse: 0.3508
Epoch 4/50
26/26 [==============================] - 0s 3ms/step - loss: 0.3160 - mae: 0.3926 - mse: 0.3160 - val_loss: 0.3758 - val_mae: 0.4073 - val_mse: 0.3758
Epoch 5/50
26/26 [==============================] - 0s 4ms/step - loss: 0.3112 - mae: 0.3812 - mse: 0.3112 - val_loss: 0.3576 - val_mae: 0.4013 - val_mse: 0.3576
Epoch 6/50
26/26 [==============================] - 0s 4ms/step - loss: 0.3073 - mae: 0.3810 - mse: 0.3073 - val_loss: 0.3524 - val_mae: 0.4007 - val_mse: 0.3524
Epoch 7/50
26/26 [=========

DEBUG flwr 2023-11-23 01:57:16,659 | connection.py:139 | gRPC channel closed
INFO flwr 2023-11-23 01:57:16,660 | app.py:215 | Disconnect and shut down


 2.3278539180755615
************MAE******************** 1.1794847249984741


In [26]:
(x_train, y_train), (x_test, y_test) = ut.partition_dataset(-2,4,3)

client = CifarClient(get_model(), x_train, y_train, x_test, y_test)

history = fl.client.start_numpy_client(
server_address="127.0.0.1:8082",
client=client,
)

INFO flwr 2023-11-23 02:00:05,368 | grpc.py:49 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-11-23 02:00:05,390 | connection.py:42 | ChannelConnectivity.IDLE
DEBUG flwr 2023-11-23 02:00:05,392 | connection.py:42 | ChannelConnectivity.READY


129/129 [==============================] - 0s 1ms/step
*************LOSS****************** 2.3105416297912598
************MAE******************** 1.1772518157958984
Epoch 1/100
26/26 [==============================] - 1s 13ms/step - loss: 0.3126 - mae: 0.3825 - mse: 0.3126 - val_loss: 0.3276 - val_mae: 0.4086 - val_mse: 0.3276
Epoch 2/100
26/26 [==============================] - 0s 3ms/step - loss: 0.3026 - mae: 0.3800 - mse: 0.3026 - val_loss: 0.3276 - val_mae: 0.4063 - val_mse: 0.3276
Epoch 3/100
26/26 [==============================] - 0s 3ms/step - loss: 0.2863 - mae: 0.3684 - mse: 0.2863 - val_loss: 0.3319 - val_mae: 0.3848 - val_mse: 0.3319
Epoch 4/100
26/26 [==============================] - 0s 3ms/step - loss: 0.2812 - mae: 0.3631 - mse: 0.2812 - val_loss: 0.3155 - val_mae: 0.3888 - val_mse: 0.3155
Epoch 5/100
26/26 [==============================] - 0s 3ms/step - loss: 0.2809 - mae: 0.3644 - mse: 0.2809 - val_loss: 0.3225 - val_mae: 0.3856 - val_mse: 0.3225
Epoch 6/100
26/26 [

DEBUG flwr 2023-11-23 02:02:12,512 | connection.py:139 | gRPC channel closed


*************LOSS****************** 2.4310128688812256
************MAE******************** 1.2073485851287842


INFO flwr 2023-11-23 02:02:12,513 | app.py:215 | Disconnect and shut down


In [ ]:
print(history)

None
